In [1]:
from selenium import webdriver
# 從selenium匯入webdriver模組
from selenium.webdriver.common.keys import Keys
# 從selenium匯入Keys來操作鍵盤(登入時會用到的Enter)
from bs4 import BeautifulSoup
# 注意英文"B"與"S"大寫
import re
# 匯入正規化模組
import time
# 匯入sleep模組
from datetime import date
# 匯入date模組>把檔名命名為抓取當日

driver = webdriver.Chrome()
# 預設爬蟲瀏覽器為Chrome(大小寫有差)
# 更多瀏覽器選擇：https://www.seleniumhq.org/download/

In [2]:
driver.get("https://pro.104.com.tw/vip/preLogin/login")
driver.implicitly_wait(5) 
# 隱性等待5秒：若瀏覽器5秒內載入完成，則繼續執行下一步驟；若5秒一到，即使未載入完成，亦強迫執行下一步
# https://huilansame.github.io/huilansame.github.io/archivers/sleep-implicitlywait-wait

driver.find_element_by_xpath("//div[@class='Login__inputField'][1]/div[@class='BaseInput']/input[@class='BaseInput__view']").send_keys('account')
driver.find_element_by_xpath("//div[@class='Login__inputField'][2]/div[@class='BaseInput']/input[@class='BaseInput__view']").send_keys('password')
driver.find_element_by_xpath("//button[@class='BaseButton']").send_keys(Keys.ENTER)
# Keys引導出後面所指定的按鍵，故須用大寫K、且鍵盤字母全為大寫，以作為區分用途(例如F5、PAGEDOWN、...)

In [ ]:
import pandas as pd
Data = pd.DataFrame(columns = ['職缺代碼', '職缺名稱', '更新日', '職務狀態', '瀏覽次數', '主動投遞數', 
                               '推薦履歷數', '職務性質', '需求人數', '職類', '職務說明', '工作待遇', 
                               '上班地點', '工業區', '管理責任', '出差外派', '上班時段', '休假制度', 
                               '可上班日', '接受身份', '工作經驗', '學歷要求', '科系要求', '語文條件', 
                               '擅長工具', '工作技能', '證照', '具備駕照', '其他條件', '職務聯絡人', 
                               '收件主管', '應徵回覆', '應徵方式', '職務關閉', '訂閱配對'])
Data

In [ ]:
for page in range(1,25):
# python記數從0開始，故page從0至21(未包含22)
    driver.get("https://pro.104.com.tw/vip/job/allJobList?page="+str(page))
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    Active = soup.find_all('td', class_='', text = re.compile('刊登中|已關閉'))
    # 職務狀態
    Date = soup.find_all('td', class_ = '', text = re.compile('\d\d/\d\d'))
    # 更新日(\d:數字)
#     for item in [Active, Date]:
#         for i in item:
#             i.extract()
            # 因60天內被瀏覽數特徵較少，會抓到其它不必要的資料
            # 以extract()方式排除list中的資料
    Job = soup.find_all('a', href = re.compile('jobmaster\?jobno='))
    # 職務名稱
    ReviewN = soup.find_all('td', class_ = '', text = re.compile('^\d+$'))
    # 60天內被瀏覽數
    ApplyN = soup.find_all('a', href = re.compile('ApplyDetail'))
    # 60天內應徵
    RecommendN = soup.find_all('a', href = re.compile('recommend/resumeListAll\?jobno='))
    # 推薦人才
    for n in range(30):
    # 對每一頁的每一個職缺抓取出所需資訊(每一頁共30筆)
    # python記數從0開始，故n從0至29(未含30)，共計29+1=30筆
        c0 = Job[n].get('href').split('=')[1]
        # 擷取職務連結代碼，以爬取個職缺內的履歷收件人
        c1 = Job[n].get_text().strip(' ')
        c2 = Date[n].get_text().strip(' ')
        c3 = Active[n].get_text().strip(' ')
        c4 = ReviewN[n].get_text().strip(' ')
        c5 = ApplyN[n].get_text().strip(' ')
        c6 = RecommendN[n].get_text().strip(' ')
        if c3 == '已關閉':
            print('已爬取完畢。')
            break
        else:
            # 以下開始抓職缺內容
            while True:
                driver.get('https://pro.104.com.tw/vip/job/jobmaster?jobno='+str(c0))
                # driver.implicitly_wait(5)
                # 停留最多5秒：讓driver讀取內容，避免爬不到資料
                time.sleep(3)
                # 隱性等待5秒還是來不及抓到資料，所以改強制等待1秒
                soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                JobDescription = soup2.find_all('div', class_='row mb-3')
                if len(JobDescription) != 0:
                    # 防止網路問題，因沒讀完網頁內容而中斷
                    break
            RedundantTitle = soup2.find_all('div', class_='col-auto col--label')
            # 移除多餘的表頭
            RedundantLink = soup2.find_all('a')
            # 移除多餘的連結(例如：市場薪資、地圖)
            RedundantLanguage = soup2.find_all('div', style='display: none;')
            # 移除語言條件中多餘的「未填寫」(已隱藏)
            RedundantComma = soup2.find_all('span', style='display: none;')
            # 移除各欄位多餘的「、」(已隱藏)
            for Redundant in [RedundantTitle, RedundantLink, RedundantLanguage, RedundantComma]:
                for r in Redundant:
                    r.extract()
            for j in range(33):
            # python記數從0開始，故i從0至30(未含31)
                globals()['JD'+str(j)] = JobDescription[j].get_text().lstrip(' ').lstrip('-')
                # globals()為以「變數」來命名變數的方法，且提供各區段的code使用
                # 相對於globals()為local()，僅限定在特定區域內(例如迴圈或函數)才有效
                # strip()為移除字首與字尾的特殊符號，若無指定則為空格(chr(32))；故lstrip為移除左邊(left)字首
            JD21RE = re.sub('<?/div>|<div.*="">','', str(JobDescription[21])).replace('<br/>',', ')
            Data = Data.append({'職缺代碼': c0, '職缺名稱': c1, '更新日':c2 , '職務狀態':c3, 
                                '瀏覽次數':c4 , '主動投遞數':c5, 
                                '推薦履歷數':c6, '職務性質': JD0, '需求人數': JD3, '職類': JD4, 
                                '職務說明': JD5, '工作待遇': JD6, '上班地點': JD8, '工業區': JD10, 
                                '管理責任': JD11, '出差外派': JD12, '上班時段': JD13, '休假制度': JD14, 
                                '可上班日': JD15, '接受身份': JD16, '工作經驗': JD17, '學歷要求': JD18, 
                                '科系要求': JD19, '語文條件': JD20, '擅長工具': JD21RE, '工作技能': JD22, 
                                '證照': JD23, '具備駕照': JD24, '其他條件': JD25, '職務聯絡人': JD26, 
                                '收件主管': JD27, '應徵回覆': JD28, '應徵方式': JD30, '職務關閉': JD31, 
                                '訂閱配對': JD32}, ignore_index=True)
                                # ignore_index=True 默認由pd預設index
            print('抓取第'+str(page)+'頁、第'+str(n)+'項')
fileName = '104Job_'+date.today().strftime('%y%m%d')+'.csv'
# 檔案名稱設定為變數，自動以當天日期來命名

Data.to_csv(fileName, encoding='utf_8_sig')
print('已爬取完畢，並將檔案名稱命名為: '+fileName)